In [2]:
import numpy as np
import pandas as pd
from stable_baselines import PPO1
from stable_baselines import DQN
from stable_baselines import A2C

/home/amirmahdi/anaconda3/envs/RL_Project2/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/home/amirmahdi/anaconda3/envs/RL_Project2/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [3]:
dataset = pd.read_csv("/home/amirmahdi/projects/Adaptive-Logging-system-git/code/data/dataframe_testing.csv")

In [5]:
DQN_model = DQN.load("DQN_model.zip")
PPO_model = PPO1.load("PPO1_model.zip")
A2C_model = A2C.load("A2C_model.zip")


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



In [21]:
def _action_determiner(obs):
        """
        The implementation logic of assiging each performance bug to a certain log level
        is based on the knowledge from previous studies mentioned above, the how common and important they are and
        also our features capabalities.
        
        arguments:
        
        obs: is supposed to be a vector (a row of the pandas dataframe);
        
        """
        #features_for_reward_func = self.features_for_reward_func
        features_for_reward_func = {
        "number_of_loops": 0,
        "nesting_level": 1,
        "input_dependant_level": 2,
        "number_of_defined_threads": 3,
        "number_of_started_threads": 4,
        "number_of_join_threads": 5,
        "number_of_defined_locks": 6,
        "number_of_acquired_locks_threads": 7,
        "number_of_released_locks": 8,
        "number_of_usage_of_extra": 9,
        "number_of_usage_of_values_list": 10,
        "number_of_usage_of_select_related": 11
    }
        #actions = self.actions
        actions = {
        "NOT_LOG": 0,
        "TRACE": 1,
        "DEBUG": 2,
        "INFO": 3,
        "WARN": 4,
        "IDK": 5
    }
        synchronization_features = obs[features_for_reward_func["number_of_defined_threads"]:
            features_for_reward_func["number_of_usage_of_extra"]]
        api_related_features = obs[features_for_reward_func["number_of_usage_of_extra"]:]
        determined_action = actions["IDK"] # Default action
        
        if (obs == 0).all(): # Checking if any of features has occured
            determined_action = actions["NOT_LOG"]
            return determined_action
        elif (synchronization_features > 0).any():
            determined_action = actions["TRACE"]
            
            obs_num_threads = obs[features_for_reward_func["number_of_defined_threads"]]
            obs_num_locks = obs[features_for_reward_func["number_of_defined_locks"]]
            obs_num_join = obs[features_for_reward_func["number_of_join_threads"]]
            race_condition_metric = obs_num_threads - obs_num_locks
            deadlock_metric = obs_num_threads - obs_num_join
            if race_condition_metric > 0 or deadlock_metric > 0: 
                determined_action = actions["DEBUG"]
            return determined_action
        else:
            nested_loop = obs[features_for_reward_func["nesting_level"]] > 0
            input_dependant = obs[features_for_reward_func["input_dependant_level"]] > 0
            if nested_loop or input_dependant:
                determined_action = actions["INFO"]
                return determined_action
            elif (api_related_features != 0).any():
                determined_action = actions["WARN"]
                return determined_action
        return determined_action

In [22]:
actions = []
d_actions = []
for record in range(len(dataset)):
    obs = dataset.iloc[record][1:].astype(np.float32)
    actions.append(DQN_model.predict(obs)[0])
    d_actions.append(_action_determiner(obs))

In [28]:
model_act = np.array(actions)
funct_act = np.array(d_actions)
model_act = model_act[model_act != 0]
funct_act = funct_act[funct_act != 0]
print(list(model_act-funct_act))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 